# `auton-survival` Cross Validation Survival Regression

`auton-survival` offers a simple to use API to train Survival Regression Models that performs cross validation model selection by minimizing integrated brier score. In this notebook we demonstrate the use of `auton-survival` to train survival models on the *SUPPORT* dataset in cross validation fashion.

In [4]:
import sys

sys.path.append('../')
from auton_survival import datasets
outcomes, features = datasets.load_support()

In [5]:
from auton_survival.preprocessing import Preprocessor

cat_feats = ['sex', 'dzgroup', 'dzclass', 'income', 'race', 'ca']
num_feats = ['age', 'num.co', 'meanbp', 'wblc', 'hrt', 'resp', 
	     'temp', 'pafi', 'alb', 'bili', 'crea', 'sod', 'ph', 
             'glucose', 'bun', 'urine', 'adlp', 'adls']

features = Preprocessor().fit_transform(features, cat_feats=cat_feats, num_feats=num_feats)

In [6]:
import numpy as np
horizons = [0.25, 0.5, 0.75]
times = np.quantile(outcomes.time[outcomes.event==1], horizons).tolist()

In [21]:
from auton_survival.experiments import SurvivalRegressionCV

param_grid = {'k' : [3],
              'distribution' : ['Weibull'],
              'learning_rate' : [ 1e-4, 1e-3],
              'layers' : [[], [100]]}

experiment = SurvivalRegressionCV(model='dsm', cv_folds=5, hyperparam_grid=param_grid, random_seed=0)
model = experiment.fit(features, outcomes)


  0%|          | 0/4 [00:00<?, ?it/s]





























 18%|█▊        | 1779/10000 [00:03<00:14, 554.25it/s]











100%|██████████| 10/10 [00:01<00:00,  7.14it/s]


























 14%|█▍        | 1418/10000 [00:02<00:17, 482.72it/s]











100%|██████████| 10/10 [00:01<00:00,  6.95it/s]





























 18%|█▊        | 1793/10000 [00:03<00:13, 588.09it/s]











100%|██████████| 10/10 [00:01<00:00,  6.14it/s]






























 18%|█▊        | 1770/10000 [00:03<00:16, 502.67it/s]











100%|██████████| 10/10 [00:01<00:00,  6.72it/s]































 18%|█▊        | 1814/10000 [00:03<00:14, 556.46it/s]











 25%|██▌       | 1/4 [00:24<01:14, 25.00s/it]


































 18%|█▊        | 1779/10000 [00:04<00:19, 428.03it/s]











100%|██████████| 10/10 [00:01<00:00,  5.74it/s]























 14%|█▍        | 1418/10000 [00:02<00:13, 618.01it/s]











100%|██████████| 10

In [20]:
model.fold

AttributeError: 'SurvivalModel' object has no attribute 'fold'

In [15]:
out_risk = model.predict_risk(features, times)
out_survival = model.predict_survival(features, times)

In [16]:
from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc

In [17]:
cis = []
brs = []

et_train = np.array([(e_train[i], t_train[i]) for i in range(len(e_train))],
                 dtype = [('e', bool), ('t', float)])
et_test = np.array([(e_test[i], t_test[i]) for i in range(len(e_test))],
                 dtype = [('e', bool), ('t', float)])
et_val = np.array([(e_val[i], t_val[i]) for i in range(len(e_val))],
                 dtype = [('e', bool), ('t', float)])

for i, _ in enumerate(times):
    cis.append(concordance_index_ipcw(et_train, et_test, out_risk[:, i], times[i])[0])
brs.append(brier_score(et_train, et_test, out_survival, times)[1])
roc_auc = []
for i, _ in enumerate(times):
    roc_auc.append(cumulative_dynamic_auc(et_train, et_test, out_risk[:, i], times[i])[0])
for horizon in enumerate(horizons):
    print(f"For {horizon[1]} quantile,")
    print("TD Concordance Index:", cis[horizon[0]])
    print("Brier Score:", brs[0][horizon[0]])
    print("ROC AUC ", roc_auc[horizon[0]][0], "\n")

ValueError: Found input variables with inconsistent numbers of samples: [1821, 9105]